# With an API key from AeroDataBox I retrieved flight arrival data, using the Cities that were defined earlier in a database on MySQLWorkbench

In [1]:
#Libraries I used
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse
from sqlalchemy import create_engine 
import mysql.connector

In [ ]:
@functions_framework.http
def test(request):
    schema = "gans_cloud" # The name of my database
    host = "REPLACEWITHIP"
    user = "root"
    password = password_GCP # My MySQL password
    port = 3306

    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    cities_df = pd.read_sql("Cities", con=connection_string)

    # Get tomorrow's date and the next 12 hours from now
    start_time = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S')
    end_time = (datetime.now() + timedelta(days=1, hours=12)).strftime('%Y-%m-%dT%H:%M:%S')

    all_flight_data = []

     # Loop through each row in the cities_df DataFrame
    for _, city in cities_df.iterrows():
        icao_code = city['Airport_Iata_code']  

        # Create the URL dynamically based on ICAO code and the start and end times
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{icao_code}/{start_time}/{end_time}"

        # Query parameters
        querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

        # Headers with the API key and host
        headers = {
            "X-RapidAPI-Key": API_flights_key,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        # Send GET request to the API
        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            # Normalize the response JSON into a pandas DataFrame
            flight_arrival_data = pd.json_normalize(response.json()['arrivals'])
            
            # Select columns and capitalize column names
            flight_arrival_data = flight_arrival_data[['number', 'status', 'departure.airport.icao', 'departure.airport.name', 'arrival.scheduledTime.local']]
            flight_arrival_data.columns = flight_arrival_data.columns.str.capitalize()

            # Add column with IATA code
            flight_arrival_data['Airport_Iata_Code'] = icao_code.upper()
        

            # Append the data to the all_flight_data list
            all_flight_data.append(flight_arrival_data)
        else:
            print(f"Error: {response.status_code}, {response.text}")

    # Concatenate all the flight data frames into one DataFrame
    if all_flight_data:
        final_flight_data = pd.concat(all_flight_data, ignore_index=True)
    else:
        print("No data to return.")
        return None

    # Replace dots with underscores in column names
    final_flight_data.columns = [col.replace('.', '_') for col in final_flight_data.columns]

    final_flight_data['Arrival_scheduledtime_local'] = pd.to_datetime(final_flight_data['Arrival_scheduledtime_local']).dt.strftime('%Y-%m-%d %H:%M:%S')


    # Then push to SQL
    final_flight_data.to_sql('Flights', con=connection_string, if_exists='append', index=False)
        
    return "Updated"  

In [ ]:
# testing
if __name__ == "__main__":
    print(fetch_and_store_weather_data())